In [7]:
import numpy as np
from scipy.sparse import rand as sprand
import torch

# Make up some random explicit feedback ratings
# and convert to a numpy array
n_users = 1_000
n_items = 1_000
ratings = sprand(n_users, n_items, density=0.01, format="csr")
ratings.data = np.random.randint(1, 5, size=ratings.nnz).astype(np.float64)
ratings = ratings.toarray()

In [64]:
ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [22]:
class BiasedMatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse=True)
        self.item_biases = torch.nn.Embedding(n_items, 1, sparse=True)

    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item)
        pred += (
            (self.user_factors(user) * self.item_factors(item))
            .sum(dim=1, keepdim=True)
        )
        return pred.squeeze()

In [40]:
model = MatrixFactorization(n_users, n_items, n_factors=64)

In [41]:
loss_func = torch.nn.MSELoss()

In [44]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)  # learning rate
adagrad_loss = torch.optim.Adagrad(model.parameters(), lr= 0.01)

In [45]:
# Sort our data
rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

num_epochs = 50
for epoch in range(num_epochs):
    for row, col in zip(*(rows, cols)):
        # Set gradients to zero
        adagrad_loss.zero_grad()
        
        # Turn data into tensors
        rating = torch.FloatTensor([ratings[row, col]])
        row = torch.LongTensor([row])
        col = torch.LongTensor([col])

        # Predict and calculate loss
        prediction = model(row, col)
        loss = loss_func(prediction, rating)

        # Backpropagate
        loss.backward()

        # Update the parameters
        adagrad_loss.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 41.934879302978516
Epoch 2/50, Loss: 38.66669464111328
Epoch 3/50, Loss: 36.47541427612305
Epoch 4/50, Loss: 34.75736618041992
Epoch 5/50, Loss: 33.31991195678711
Epoch 6/50, Loss: 32.07292938232422
Epoch 7/50, Loss: 30.965871810913086
Epoch 8/50, Loss: 29.9670352935791
Epoch 9/50, Loss: 29.05497169494629
Epoch 10/50, Loss: 28.214448928833008
Epoch 11/50, Loss: 27.43416976928711
Epoch 12/50, Loss: 26.70545196533203
Epoch 13/50, Loss: 26.021512985229492
Epoch 14/50, Loss: 25.376890182495117
Epoch 15/50, Loss: 24.767105102539062
Epoch 16/50, Loss: 24.18848991394043
Epoch 17/50, Loss: 23.6379451751709
Epoch 18/50, Loss: 23.112844467163086
Epoch 19/50, Loss: 22.610910415649414
Epoch 20/50, Loss: 22.130212783813477
Epoch 21/50, Loss: 21.66907501220703
Epoch 22/50, Loss: 21.225967407226562
Epoch 23/50, Loss: 20.799579620361328
Epoch 24/50, Loss: 20.388742446899414
Epoch 25/50, Loss: 19.992414474487305
Epoch 26/50, Loss: 19.609697341918945
Epoch 27/50, Loss: 19.2396945953369

In [63]:
movies_not_rated = ratings[~ratings['Movie_Id'].isin(ratings[ratings['User_Id'] == 1]['Movie_Id'])]['Movie_Id']
movies_not_rated

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
user_id_to_recommend = 1
movies_not_rated = ratings.loc[~ratings['Movie_Id'].isin(ratings[ratings['User_Id'] == user_id_to_recommend]['Movie_Id'])]['Movie_Id']
user_movie_input = torch.LongTensor([[user_id_to_recommend, movie_id] for movie_id in movies_not_rated])

model.eval()
with torch.no_grad():
    predictions = model(user_movie_input).numpy()

recommendations = pd.DataFrame({'Movie_Id': movies_not_rated, 'Predicted_Rating': predictions})
top_recommendations = recommendations.nlargest(5, 'Predicted_Rating')

print(f"Top Recommendations for User {user_id_to_recommend}:")
for idx, row in top_recommendations.iterrows():
    movie_id = int(row['Movie_Id'])
    print(f"Movie ID: {movie_id}")

In [9]:
import numpy as np
from scipy.sparse.linalg import svds

# Create a user-item matrix
a = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])
a = a.astype(float)

# Perform matrix factorization using Singular Value Decomposition (SVD)
U, sigma, Vt = svds(a, k=2)

print("U (user features) = ")
print(U)
print("\nVt (item features) = ")
print(Vt)
print("\nSigma (sigma features) = ")
print(sigma)

print(np.dot(U, Vt))

U (user features) = 
[[-0.66924125 -0.43689593]
 [-0.44308727 -0.29717498]
 [ 0.13631518 -0.51589728]
 [ 0.11077382 -0.39999635]
 [ 0.5700326  -0.54282768]]

Vt (item features) = 
[[-0.78203025 -0.20891356  0.45754472  0.36801718]
 [-0.47488998 -0.26234348 -0.3005118  -0.78444124]]

Sigma (sigma features) = 
[6.22925557 9.03171974]
[[ 0.7308444   0.25443037 -0.17491541  0.09642691]
 [ 0.48763307  0.17052886 -0.11342765  0.07005259]
 [ 0.13839186  0.1068642   0.21740351  0.45485743]
 [ 0.10332578  0.08179428  0.1708876   0.35454031]
 [-0.18799931  0.02331976  0.42394153  0.63559821]]


In [ ]:
prediction = [2.67366361618042, 2.7077696323394775, 2.5114035606384277, 2.647676944732666, 2.6722922325134277, 2.703892230987549, 2.71844744682312, 2.7034473419189453, 2.624603033065796, 2.654174327850342, 2.670908212661743, 2.6982617378234863, 2.710632801055908, 2.6590704917907715, 2.63384747505188, 2.6758687496185303, 2.6761252880096436, 2.650397300720215, 2.7085659503936768, 2.6319615840911865, 2.6686511039733887, 2.699941873550415, 2.6931169033050537, 2.6872472763061523, 2.6980085372924805, 2.6647536754608154, 2.680485725402832, 2.6303322315216064, 2.704845666885376, 2.697699785232544, 2.7228546142578125, 2.7579588890075684, 2.6906299591064453, 2.654094934463501, 2.726816177368164, 2.677264928817749, 2.698096513748169, 2.6784920692443848, 2.6980984210968018, 2.6792311668395996, 2.6720242500305176, 2.6463799476623535, 2.539799928665161, 2.659109354019165, 2.678255558013916, 2.712296962738037, 2.6761631965637207, 2.678703546524048, 2.630354166030884, 2.7007851600646973, 2.688063144683838, 2.674983024597168, 2.7021002769470215, 2.6871962547302246, 2.688302516937256, 2.659937858581543, 2.7320806980133057, 2.6836249828338623, 2.6775636672973633, 2.7027785778045654, 2.694517135620117, 2.7094075679779053, 2.6935739517211914, 2.715378522872925, 2.6628847122192383, 2.687129020690918, 2.6996850967407227, 2.675323963165283, 2.7291452884674072, 2.632388114929199, 2.6974194049835205, 2.676706075668335, 2.6713380813598633, 2.6999194622039795, 2.6404404640197754, 2.666600227355957, 2.6804115772247314, 2.698472738265991, 2.695831298828125, 2.66030216217041, 2.6940066814422607, 2.685145139694214, 2.677706241607666, 2.622281551361084, 2.6698477268218994, 2.69063663482666, 2.6929874420166016, 2.721238374710083, 2.729372024536133, 2.720093011856079, 2.7033350467681885, 2.6927056312561035, 2.6793577671051025, 2.718799352645874, 2.6941566467285156, 2.658179759979248, 2.7152957916259766, 2.671743392944336, 2.6894924640655518, 2.7137327194213867, 2.6997129917144775, 2.705461263656616, 2.7147116661071777, 2.637444496154785, 2.7038605213165283, 2.705786943435669, 2.6546506881713867, 2.648205041885376, 2.7134947776794434, 2.7367310523986816, 2.7252087593078613, 2.721186399459839, 2.6836624145507812, 2.704105854034424, 2.7003564834594727, 2.6627111434936523, 2.6177544593811035, 2.6700572967529297, 2.676161527633667, 2.6950674057006836, 2.707711696624756, 2.6910927295684814, 2.673523426055908, 2.654798984527588, 2.678213357925415, 2.7462663650512695, 2.4850990772247314, 2.6743180751800537, 2.6819915771484375, 2.709099054336548, 2.660295009613037, 2.6706881523132324, 2.6786298751831055, 2.720895290374756, 2.6271936893463135, 2.7026174068450928, 2.6837480068206787, 2.694415807723999, 2.6892786026000977, 2.717386484146118, 2.701385021209717, 2.6694960594177246, 2.6514170169830322, 2.684507369995117, 2.633509397506714, 2.693301200866699, 2.653118133544922, 2.665048122406006, 2.6788692474365234, 2.481450319290161, 2.6419756412506104, 2.6888039112091064, 2.705007553100586, 2.6932461261749268, 2.6736598014831543, 2.647319793701172, 2.7057032585144043, 2.7323198318481445, 2.65313720703125, 2.6679961681365967, 2.7007410526275635, 2.716851234436035, 2.647714853286743, 2.6540133953094482, 2.658503293991089, 2.679772138595581, 2.6933741569519043, 2.560046911239624, 2.621581792831421, 2.7434003353118896, 2.730292558670044, 2.6993913650512695, 2.6528592109680176, 2.6887097358703613, 2.69111704826355, 2.694702625274658, 2.69876766204834, 2.6844639778137207, 2.7080490589141846, 2.6836087703704834, 2.672182321548462, 2.656313419342041, 2.6653623580932617, 2.671560287475586, 2.6837432384490967, 2.720567226409912, 2.7132623195648193, 2.4533917903900146, 2.710405111312866, 2.661330223083496, 2.6965794563293457, 2.67172908782959, 2.660508871078491, 2.6249475479125977, 2.6643567085266113, 2.7181034088134766, 2.65547776222229, 2.653477668762207, 2.6994452476501465, 2.7089269161224365, 2.7001023292541504, 2.7188026905059814, 2.6466856002807617, 2.711681365966797, 2.6889610290527344, 2.672935962677002, 2.665393829345703, 2.6928164958953857, 2.663426399230957, 2.695958137512207, 2.66744065284729, 2.728031635284424, 2.6752140522003174, 2.703787326812744, 2.69425892829895, 2.66192364692688, 2.6818299293518066, 2.6797704696655273, 2.686099052429199, 2.6735100746154785, 2.6934618949890137, 2.6897993087768555, 2.741115093231201, 2.6721537113189697, 2.639362335205078, 2.6712515354156494, 2.6874821186065674, 2.675436496734619, 2.696582317352295, 2.6987967491149902, 2.711440324783325, 2.659987688064575, 2.65950870513916, 2.6889781951904297, 2.6690163612365723, 2.697455406188965, 2.7216858863830566, 2.556483030319214, 2.6977591514587402, 2.7148566246032715, 2.7606658935546875, 2.7027909755706787, 2.6833372116088867, 2.7044565677642822, 2.6716806888580322, 2.6604199409484863, 2.6454830169677734, 2.6671276092529297, 2.6131601333618164, 2.6388375759124756, 2.67154598236084, 2.650876760482788, 2.6705374717712402, 2.6778602600097656, 2.7030868530273438, 2.687472343444824, 2.7388737201690674, 2.6737332344055176, 2.6874711513519287, 2.665599822998047, 2.6846628189086914, 2.7062907218933105, 2.7137091159820557, 2.716817855834961, 2.7018373012542725, 2.614532470703125, 2.6988139152526855, 2.702068328857422, 2.6314735412597656, 2.6977601051330566, 2.6752281188964844, 2.6739208698272705, 2.6718432903289795, 2.6853392124176025, 2.644404888153076, 2.6905901432037354, 2.7374801635742188, 2.704988956451416, 2.6878161430358887, 2.5226664543151855, 2.7298600673675537, 2.713378667831421, 2.601991891860962, 2.6977813243865967, 2.6818597316741943, 2.661818265914917, 2.64910626411438, 2.624501943588257, 2.662632942199707, 2.680036783218384, 2.685405731201172, 2.7111520767211914, 2.6708731651306152, 2.709263324737549, 2.7034339904785156, 2.6844067573547363, 2.680387496948242, 2.705472469329834, 2.706324577331543, 2.6736037731170654, 2.7022688388824463, 2.389359712600708, 2.703036308288574, 2.66660737991333, 2.7002193927764893, 2.6975038051605225, 2.685563564300537, 2.6887927055358887, 2.6687521934509277, 2.701350688934326, 2.7091870307922363, 2.676333427429199, 2.6294069290161133, 2.631328582763672, 2.658557653427124, 2.657904863357544, 2.684074878692627, 2.7103829383850098, 2.714674949645996, 2.711839437484741, 2.734205961227417, 2.7033021450042725, 2.6501035690307617, 2.6432878971099854, 2.702399492263794, 2.671475410461426, 2.6598687171936035, 2.7041821479797363, 2.689462423324585, 2.739241600036621, 2.6676552295684814, 2.6232757568359375, 2.651717185974121, 2.652587890625, 2.68466854095459, 2.6641793251037598, 2.6980133056640625, 2.6856818199157715, 2.6743528842926025, 2.64421010017395, 2.6552486419677734, 2.6774137020111084, 2.698528289794922, 2.6876354217529297, 2.70664119720459, 2.716622829437256, 2.736908435821533, 2.706364870071411, 2.633829116821289, 2.6675543785095215, 2.7065253257751465, 2.7116782665252686, 2.636972427368164, 2.663418769836426, 2.66839599609375, 2.6964993476867676, 2.6728515625, 2.6936378479003906, 2.6795246601104736, 2.7307887077331543, 2.6704814434051514, 2.7188072204589844, 2.685701370239258, 2.7125792503356934, 2.6556525230407715, 2.6940016746520996, 2.67781925201416, 2.6988182067871094, 2.6748404502868652, 2.6608049869537354, 2.6162846088409424, 2.7414073944091797, 2.6732358932495117, 2.712669849395752, 2.6724812984466553, 2.7004168033599854, 2.6431846618652344, 2.6898550987243652, 2.7160732746124268, 2.7269110679626465, 2.679757833480835, 2.6604716777801514, 2.689295530319214, 2.6699087619781494, 2.6657238006591797, 2.6586203575134277, 2.717707633972168, 2.7343358993530273, 2.6636838912963867, 2.7214128971099854, 2.6877665519714355, 2.6343212127685547, 2.674501895904541, 2.6623694896698, 2.623694896697998, 2.712581157684326, 2.7416319847106934, 2.666706085205078, 2.628740072250366, 2.722656488418579, 2.6723992824554443, 2.7082290649414062, 2.678070545196533, 2.638465642929077, 2.679157018661499, 2.679360866546631, 2.693023681640625, 2.6802315711975098, 2.7081375122070312, 2.6519949436187744, 2.6632418632507324, 2.678527355194092, 2.660372257232666, 2.6610631942749023, 2.648686170578003, 2.6072192192077637, 2.6568422317504883, 2.6523847579956055, 2.6929306983947754, 2.661402940750122, 2.7050933837890625, 2.7181901931762695, 2.492042064666748, 2.7157979011535645, 2.670036554336548, 2.6739957332611084, 2.649773597717285, 2.7005162239074707, 2.6946096420288086, 2.7335290908813477, 2.3112730979919434, 2.7237205505371094, 2.6664834022521973, 2.7209572792053223, 2.6506872177124023, 2.693094491958618, 2.701277017593384, 2.675847053527832, 2.7230708599090576, 1.6542041301727295, 2.7229132652282715, 2.677417278289795, 2.6630876064300537, 2.6398892402648926, 2.726193428039551, 2.62294340133667, 2.659438371658325, 2.6970551013946533, 2.701876640319824, 2.7114737033843994, 2.641836166381836, 2.6462290287017822, 2.674034833908081, 2.68083119392395, 2.69844913482666, 2.712337017059326, 2.6706507205963135, 2.6772708892822266, 2.679063081741333, 2.593576192855835, 2.6921350955963135, 2.6275863647460938, 2.6794137954711914, 2.6386356353759766, 2.6472277641296387, 2.6821177005767822, 2.6664929389953613, 2.590704917907715, 2.6073105335235596, 2.6530730724334717, 2.7072277069091797, 2.6891684532165527, 2.6633291244506836, 2.6870675086975098, 2.651078939437866, 2.6999096870422363, 2.674267292022705, 2.639739990234375, 2.62695574760437, 2.6374003887176514, 2.695052146911621, 2.6425461769104004, 2.7394680976867676, 2.656035900115967, 2.5927810668945312, 2.6827993392944336, 2.6268062591552734, 2.6755802631378174, 2.717740535736084, 2.678179979324341, 2.661184549331665, 2.7265660762786865, 2.724247455596924, 2.359750986099243, 2.6927170753479004, 2.6690454483032227, 2.6932730674743652, 2.6756415367126465, 2.6912288665771484, 2.6709423065185547, 2.712867259979248, 2.6958422660827637, 2.667863130569458, 2.6744327545166016, 2.6866326332092285, 2.7395520210266113, 2.7171144485473633, 2.5834097862243652, 2.740330457687378, 2.694037675857544, 2.6299121379852295, 2.6536571979522705, 2.767113447189331, 2.6635143756866455, 2.61641788482666, 2.6809799671173096, 2.6441445350646973, 2.698864221572876, 2.661165237426758, 2.6714911460876465, 2.6774048805236816, 2.639702796936035, 2.7440621852874756, 2.65635347366333, 2.6117470264434814, 2.647928237915039, 2.6671102046966553, 2.617565393447876, 2.667936086654663, 2.658108711242676, 2.6493444442749023, 2.6564598083496094, 2.6931588649749756, 2.724914312362671, 2.662982225418091, 2.689260721206665, 2.7013299465179443, 2.6740803718566895, 2.6942830085754395, 2.6646294593811035, 2.6835391521453857, 2.7329580783843994, 2.561851978302002, 2.667797088623047, 2.700932741165161, 1.6508573293685913, 2.702866792678833, 2.6869587898254395, 2.678595781326294, 2.6690030097961426, 2.661059617996216, 2.679812431335449, 2.6700656414031982, 2.708618640899658, 2.7103848457336426, 2.6381821632385254, 2.712810516357422, 2.7166085243225098, 2.5949015617370605, 2.6440610885620117, 2.657414197921753, 2.7245335578918457, 2.7073299884796143, 2.717195987701416, 2.6474485397338867, 2.7196974754333496, 2.6767568588256836, 2.696523427963257, 2.696760416030884, 2.706876039505005, 2.630957841873169, 2.7284817695617676, 2.65937876701355, 2.698394775390625, 2.6823177337646484, 2.6377289295196533, 2.6147730350494385, 2.6612448692321777, 2.618730068206787, 2.6493849754333496, 2.6328272819519043, 2.707094192504883, 2.6617789268493652, 2.6363906860351562, 2.7262330055236816, 2.641544818878174, 2.627375602722168, 2.685826063156128, 2.6795952320098877, 2.6551403999328613, 2.687540292739868, 2.672323226928711, 2.6979215145111084, 2.697253704071045, 2.6457290649414062, 2.719194173812866, 2.65962815284729, 2.6633424758911133, 2.611834764480591, 2.7260148525238037, 2.6853344440460205, 2.714263439178467, 2.6621687412261963, 2.703251361846924, 2.6442155838012695, 2.626960277557373, 2.701699733734131, 2.6760013103485107, 2.707524299621582, 2.6401078701019287, 2.679384708404541, 2.6906516551971436, 2.7102818489074707, 2.7102832794189453, 2.676867961883545, 2.685331106185913, 2.669149160385132, 2.711792469024658, 2.725630760192871, 2.696122169494629, 2.6696105003356934, 2.635157585144043, 2.6910297870635986, 2.7027134895324707, 1.6752396821975708, 2.7088122367858887, 2.6046431064605713, 2.5532100200653076, 2.5900237560272217, 2.713168144226074, 2.7027900218963623, 2.6647305488586426, 2.7314095497131348, 2.6920859813690186, 2.1519649028778076, 2.6849465370178223, 2.6824028491973877, 2.680410861968994, 2.7331526279449463, 1.7125641107559204, 2.6649329662323, 2.6056370735168457, 2.6567742824554443, 2.7266933917999268, 2.6775765419006348, 2.624537467956543, 2.7269287109375, 2.677985668182373, 2.677405834197998, 2.7093331813812256, 2.2705748081207275, 2.6917920112609863, 2.7286932468414307, 2.6703338623046875, 2.696746587753296, 2.6253957748413086, 2.683300733566284, 2.7245712280273438, 2.6971030235290527, 2.5790724754333496, 2.6872034072875977, 2.6747589111328125, 2.663269519805908, 2.656895637512207, 2.6751043796539307, 2.6532058715820312, 2.655886650085449, 2.666501998901367, 2.6277530193328857, 2.664320468902588, 2.7054078578948975, 2.636627674102783, 2.705702304840088, 2.7007880210876465, 2.500950813293457, 2.670116662979126, 2.6670589447021484, 2.669433116912842, 2.6534481048583984, 2.6307590007781982, 2.7555222511291504, 2.7202260494232178, 2.6497788429260254, 2.6694936752319336, 2.651838779449463, 2.6537764072418213, 2.6878111362457275, 2.672941207885742, 2.714940309524536, 2.6038424968719482, 2.678410768508911, 2.683213710784912, 2.7031803131103516, 2.6902549266815186, 2.699254035949707, 2.6889333724975586, 2.617506742477417, 2.70378041267395, 2.6762211322784424, 2.7071008682250977, 2.651106834411621, 2.5448880195617676, 2.684953451156616, 2.693239450454712, 2.618471145629883, 2.672398567199707, 2.7006642818450928, 2.647592067718506, 2.652165412902832, 2.666971206665039, 2.647589683532715, 2.6118342876434326, 2.688629627227783, 2.682584285736084, 2.62786602973938, 2.6400492191314697, 2.670198678970337, 2.6875545978546143, 2.690654754638672, 2.696925640106201, 2.709567070007324, 2.6949498653411865, 2.6997954845428467, 2.6883535385131836, 2.5705654621124268, 2.694739580154419, 2.689804792404175, 2.6555347442626953, 2.6620912551879883, 2.7507171630859375, 2.7093889713287354, 2.5900416374206543, 2.6806859970092773, 2.685415267944336, 2.6573450565338135, 2.6823792457580566, 2.6683521270751953, 2.6631085872650146, 2.623161554336548, 2.667215585708618, 2.7024002075195312, 2.628594160079956, 2.674393653869629, 2.655616283416748, 2.693235397338867, 2.6580729484558105, 2.7053229808807373, 2.680889844894409, 2.7006449699401855, 2.6351535320281982, 2.6966805458068848, 2.7043912410736084, 2.5406296253204346, 2.674837589263916, 2.6587166786193848, 2.6541171073913574, 2.615339517593384, 2.6550753116607666, 2.706225633621216, 2.6003212928771973, 2.68155574798584, 2.6226227283477783, 2.698108673095703, 2.676633834838867, 2.6156911849975586, 2.6378395557403564, 2.6642844676971436, 2.685384511947632, 2.489664316177368, 2.627870559692383, 2.7051024436950684, 2.611710548400879, 2.6684656143188477, 2.6270437240600586, 2.6209230422973633, 2.65620493888855, 2.445831298828125, 2.6356849670410156, 2.6660704612731934, 2.675590991973877, 2.735746145248413, 2.6815927028656006, 2.6507084369659424, 2.644822359085083, 2.6937801837921143, 2.679741859436035, 2.677830457687378, 2.6027657985687256, 2.62733793258667, 2.6458981037139893, 2.679032802581787, 2.596649169921875, 2.565589666366577, 2.3693768978118896, 2.6906633377075195, 2.7116899490356445, 2.692671775817871, 2.670161485671997, 2.6800899505615234, 2.6577162742614746, 2.5979769229888916, 2.6794750690460205, 2.6821963787078857, 2.65022873878479, 2.6321887969970703, 2.612413167953491, 2.6991193294525146, 2.7110986709594727, 2.68027663230896, 2.6575284004211426, 2.7158148288726807, 2.7067618370056152, 2.6731173992156982, 2.4758529663085938, 2.6471052169799805, 2.643049955368042, 2.696359634399414, 2.6768388748168945, 2.70289945602417, 2.6998422145843506, 2.659372091293335, 2.7288262844085693, 2.7285585403442383, 2.645491600036621, 2.660400152206421, 2.603224992752075, 2.6038811206817627, 2.6549997329711914, 2.672832489013672, 2.650974750518799, 2.61859393119812, 2.633664608001709, 2.62319278717041, 2.6366920471191406, 2.648465394973755, 2.671628952026367, 2.5679593086242676, 2.6819655895233154, 2.677565336227417, 2.680481433868408, 2.6532061100006104, 2.6447622776031494, 2.71433162689209, 2.6775505542755127, 2.68792724609375, 1.7089210748672485, 2.637314558029175, 2.5389764308929443, 2.636774778366089, 2.638580322265625, 2.434751272201538, 2.7104227542877197, 2.6840150356292725, 2.6894235610961914, 2.5508856773376465, 2.6713218688964844, 2.6906185150146484, 2.6752562522888184, 2.6669821739196777, 2.6773321628570557, 2.655900239944458, 2.667161464691162, 2.6830379962921143, 2.670933485031128, 2.6793723106384277, 2.630246639251709, 2.718510866165161, 2.7080399990081787, 2.642967462539673, 2.4669175148010254, 2.670851469039917, 0.30972275137901306, 2.6718435287475586, 2.678776741027832, 2.6767351627349854, 2.6823174953460693, 2.6571435928344727, 2.695430278778076, 2.571321487426758, 2.4849843978881836, 2.6427276134490967, 2.6067869663238525, 2.6638903617858887, 2.680485248565674, 2.683436870574951, 2.6412158012390137, 2.675335645675659, 2.6999433040618896, 2.7314083576202393, 2.7123517990112305, 2.6481783390045166, 2.6881103515625, 2.6233386993408203, 2.551591634750366, 2.643739700317383, 2.695650100708008, 2.67475962638855, 2.644456624984741, 2.6433143615722656, 2.6023731231689453, 2.6851003170013428, 2.6100027561187744, 2.7280161380767822, 2.6605424880981445, 2.6584439277648926, 2.6880247592926025, 2.664187431335449, 2.711652994155884, 2.7018375396728516, 1.6614775657653809, 2.664099931716919, 2.6240625381469727, 2.6780505180358887, 2.6713593006134033, 2.7055158615112305, 2.701267719268799, 2.6919450759887695, 1.6669137477874756, 2.659003496170044, 2.6635944843292236, 2.6826725006103516, 2.6537327766418457, 2.6740474700927734, 2.699981212615967, 2.5764803886413574, 2.657804250717163, 2.6658453941345215, 2.6019740104675293, 2.644868850708008, 1.7073414325714111, 2.6398258209228516, 2.6710758209228516, 2.5940043926239014, 2.5843725204467773, 2.638380527496338, 2.702141523361206, 2.685356855392456, 2.6054859161376953, 2.6814496517181396, 2.7103397846221924, 2.69012451171875, 0.30393385887145996, 2.696071147918701, 2.7054505348205566, 2.6603240966796875, 2.6396121978759766, 2.6041409969329834, 2.661015272140503, 2.7247862815856934, 2.676785945892334, 2.725209951400757, 2.668712854385376, 2.666888952255249, 2.663081169128418, 2.688823699951172, 2.655590534210205, 2.674072504043579, 2.720065116882324, 2.695467472076416, 2.7119672298431396, 2.6462817192077637, 2.671339511871338, 2.5893943309783936, 2.6432714462280273, 2.6671133041381836, 2.70058274269104, 2.554551362991333, 2.118438243865967, 2.608642578125, 2.7382729053497314, 2.6827664375305176, 2.668882131576538, 2.7133631706237793, 2.676135778427124, 2.6565442085266113, 2.5915942192077637, 2.682206392288208, 2.6935441493988037, 2.6422533988952637, 2.6902265548706055, 2.655092239379883, 2.659437417984009, 2.7002086639404297, 2.684988260269165, 2.642117977142334, 2.514613389968872, 2.522524356842041, 2.6867258548736572, 2.6571195125579834, 2.6794517040252686, 2.6491637229919434, 2.712790012359619, 2.621183395385742, 2.685192108154297, 2.500138759613037, 2.6123604774475098, 2.544114589691162, 2.705430746078491, 2.6080071926116943, 2.70607328414917, 2.705691337585449, 2.6783218383789062, 2.6162734031677246, 2.677668571472168, 2.6334187984466553, 2.7164487838745117, 2.674318790435791, 2.6652867794036865, 2.6588597297668457, 2.704821825027466, 2.612837553024292, 2.4594388008117676, 2.6631884574890137, 2.557542324066162, 2.6920578479766846, 2.6662416458129883, 2.6744847297668457, 2.4720234870910645, 2.6552934646606445, 2.6626410484313965, 2.681056261062622, 2.6777493953704834, 2.6819868087768555, 2.653932809829712, 2.7135884761810303, 2.4923815727233887, 2.666120767593384, 2.580976724624634, 2.7197678089141846, 2.646371841430664, 2.6129698753356934, 2.642089366912842, 2.6329174041748047, 1.6358373165130615, 2.706963062286377, 2.4008779525756836, 2.6870646476745605, 2.698664665222168, 2.738616466522217, 2.6396148204803467, 2.6730830669403076, 2.633265733718872, 2.6644093990325928, 2.59238338470459, 2.589127540588379, 2.522728681564331, 2.6978816986083984, 2.6646761894226074, 2.6431171894073486, 2.665559768676758, 1.633206844329834, 2.6962156295776367, 2.6112663745880127, 2.6679491996765137, 2.667588949203491, 2.668996810913086, 2.7209627628326416, 2.6482183933258057, 2.6468420028686523, 2.698280096054077, 2.7014265060424805, 2.6955995559692383, 2.492466688156128, 2.72188663482666, 2.587939739227295, 2.7033207416534424, 2.657522439956665, 2.7239890098571777, 2.64140248298645, 2.670905113220215, 2.6428208351135254, 2.672729730606079, 2.7262840270996094, 2.6001672744750977, 2.7175052165985107, 1.62076997756958, 2.684664249420166, 2.613079786300659, 2.6588618755340576, 2.6864542961120605, 2.37131404876709, 2.6350715160369873, 2.6277968883514404, 2.6248204708099365, 2.6785712242126465, 2.5692639350891113, 2.6192870140075684, 0.2829558253288269, 2.6443634033203125, 2.6308562755584717, 2.6805031299591064, 2.6348876953125, 2.6443445682525635, 2.3793396949768066, 2.7369723320007324, 2.645810604095459, 2.642414093017578, 2.723043918609619, 2.712038516998291, 2.4328367710113525, 2.5358352661132812, 2.6061325073242188, 2.677874803543091, 2.6199605464935303, 2.671457052230835, 2.645714044570923, 2.6676244735717773, 2.7088303565979004, 2.692430257797241, 2.59192156791687, 2.6227011680603027, 2.687917947769165, 2.6601364612579346, 2.6743083000183105, 2.674649953842163, 2.6934914588928223, 2.6259846687316895, 2.6795201301574707, 2.678915023803711, 2.6661622524261475, 2.6840295791625977, 2.64770770072937, 2.677993059158325, 2.6521220207214355, 2.648810625076294, 2.624976873397827, 2.6832518577575684, 2.6996655464172363, 2.6755123138427734, 2.5530223846435547, 2.6723551750183105, 2.6400363445281982, 2.5078821182250977, 2.6961238384246826, 2.704970359802246, 2.6720023155212402, 2.6164586544036865, 2.6725194454193115, 2.6779279708862305, 2.644852876663208, 1.6452858448028564, 2.5398881435394287, 2.6839311122894287, 2.665188789367676, 2.5654003620147705, 2.6782755851745605, 2.643707036972046, 1.7062687873840332, 2.660287380218506, 2.669952392578125, 2.6798834800720215, 2.6344940662384033, 2.616656541824341, 2.117318630218506, 2.618706703186035, 2.7045950889587402, 2.6806817054748535, 2.606809377670288, 2.5672800540924072, 2.535794496536255, 2.6859655380249023, 2.670058012008667, 2.659008502960205, 2.6800994873046875, 2.663456678390503, 2.708157777786255, 2.668045997619629, 2.6725614070892334, 2.7011396884918213, 2.6721529960632324, 2.6343307495117188, 2.6969361305236816, 2.641861915588379, 1.656008005142212, 2.6775121688842773, 2.5978574752807617, 2.6725330352783203, 2.7258827686309814, 2.679903984069824, 2.660172939300537, 2.7283670902252197, 2.6029415130615234, 2.657308340072632, 2.5558204650878906, 2.678250789642334, 2.6241073608398438, 2.5662753582000732, 2.6702098846435547, 2.7171220779418945, 2.486971855163574, 2.663587808609009, 2.6209185123443604, 2.6916019916534424, 2.5813698768615723, 2.6200788021087646, 2.7041378021240234, 2.7027859687805176, 2.56484317779541, 2.6466128826141357, 2.6994290351867676, 2.683415174484253, 2.6687493324279785, 2.567685127258301, 2.6724236011505127, 2.674778938293457, 2.647364377975464, 2.6910061836242676, 2.708582878112793, 1.6379947662353516, 2.603278160095215, 2.6036548614501953, 2.671198844909668, 2.605631113052368, 2.5803489685058594, 2.658928394317627, 2.702995777130127, 2.615915536880493, 2.680651903152466, 1.6401872634887695, 2.695857048034668, 2.701068639755249, 1.7086856365203857, 2.5457119941711426, 2.650752067565918, 2.629352331161499, 2.6572182178497314, 2.6762726306915283, 2.6618659496307373, 2.5472753047943115, 2.6605465412139893, 2.7030715942382812, 2.6358182430267334, 2.627060651779175, 2.6109206676483154, 2.4155921936035156, 2.684997797012329, 2.6521172523498535, 2.654083013534546, 2.5774435997009277, 2.7096807956695557, 2.614332675933838, 2.656395673751831, 1.666764736175537, 2.681575298309326, 2.6598081588745117, 2.675701379776001, 2.594064474105835, 2.657426118850708, 2.676819324493408, 2.614670991897583, 2.666125535964966, 2.6016902923583984, 2.70418119430542, 2.635422945022583, 2.5532891750335693, 2.733427047729492, 2.6640877723693848, 2.437431812286377, 2.5857882499694824, 2.7005069255828857, 2.633272171020508, 2.499936103820801, 2.405604362487793, 2.6084036827087402, 2.650421142578125, 2.653501510620117, 2.4558427333831787, 2.3820674419403076, 2.5412018299102783, 2.5904476642608643, 2.6473119258880615, 2.7283105850219727, 1.640045166015625, 2.614276885986328, 2.668775796890259, 2.6510090827941895, 2.6632802486419678, 2.6331632137298584, 2.4407925605773926, 2.4929726123809814, 2.610323429107666, 2.669306755065918, 2.7023890018463135, 1.6771355867385864, 2.6781537532806396, 2.7217230796813965, 2.683806896209717, 2.691225528717041, 2.6740689277648926, 2.509204864501953, 2.618013381958008, 2.636565923690796, 2.6920502185821533, 2.5343000888824463, 2.723173141479492, 2.6949095726013184, 2.5040345191955566, 0.29988083243370056, 0.2572483718395233, 1.7075389623641968, 2.6332242488861084, 2.663025140762329, 2.530609130859375, 2.528440475463867, 0.2821351885795593, 1.6685779094696045, 2.6820778846740723, 2.2558655738830566, 2.6740071773529053, 2.5856900215148926, 2.7048773765563965, 1.6474860906600952, 2.697096824645996, 2.7032177448272705, 2.6276416778564453, 2.6498594284057617, 2.6064107418060303, 2.679872751235962, 2.603100299835205, 0.28257814049720764, 2.6941094398498535, 1.640616536140442, 2.72686767578125, 2.748335838317871, 2.6696040630340576, 2.6640663146972656, 2.651118040084839, 2.6389615535736084, 2.6619997024536133, 2.6676838397979736, 2.670867443084717, 2.554814577102661, 1.6916459798812866, 2.6773436069488525, 2.7072174549102783, 1.6426432132720947, 2.674583673477173, 2.533527374267578, 2.672595977783203, 2.100170612335205, 2.674696445465088, 2.644794464111328, 1.6448010206222534, 2.6534440517425537, 2.6806936264038086, 2.256239891052246, 2.5604701042175293, 2.69856595993042, 2.6245219707489014, 2.6849939823150635, 2.67122220993042, 2.4964754581451416, 2.623772144317627, 2.6338119506835938, 2.7245821952819824, 2.6429123878479004, 2.64152455329895, 2.6714017391204834, 2.5090537071228027, 2.672731876373291, 2.6483356952667236, 2.6712231636047363, 2.57798433303833, 2.6053733825683594, 2.532597064971924, 2.659680128097534, 2.6991114616394043, 2.639347553253174, 1.6444875001907349, 2.6353437900543213, 2.648400068283081, 2.679940938949585, 2.6438870429992676, 2.697082996368408, 2.294363260269165, 2.6695058345794678, 2.62554931640625, 2.669332265853882, 2.6845903396606445, 2.6786932945251465, 2.697312116622925, 2.557053327560425, 2.627305507659912, 2.5635600090026855, 2.700021982192993, 2.700571298599243, 2.5819809436798096, 2.6902122497558594, 1.6365524530410767, 2.660546064376831, 2.686572790145874, 2.658963918685913, 2.575413227081299, 1.6434475183486938, 2.7080726623535156, 1.6551634073257446, 0.30056214332580566, 2.693389654159546, 2.673854112625122, 2.7024364471435547, 2.657984495162964, 2.683077812194824, 2.701871395111084, 2.6546316146850586, 0.30141741037368774, 2.671433687210083, 2.6480705738067627, 2.4722180366516113, 2.6058411598205566, 2.694025993347168, 2.7272889614105225, 2.632625102996826, 2.5150628089904785, 2.572652578353882, 2.716639757156372, 1.684701681137085, 2.6542139053344727, 2.668769121170044, 0.30903810262680054, 2.687769889831543, 1.705729603767395, 2.6560781002044678, 2.654968500137329, 1.6691207885742188, 1.7082798480987549, 2.677288055419922, 2.6673645973205566, 2.630021095275879, 2.6732568740844727, 2.1877760887145996, 2.668090581893921, 1.639837384223938, 2.6492557525634766, 2.6776750087738037, 2.6410179138183594, 2.6507723331451416, 2.696690559387207, 1.7097187042236328, 2.5235066413879395, 2.5482065677642822, 0.29696422815322876, 0.3343105614185333, 2.6801323890686035, 2.528191328048706, 2.6799230575561523, 1.996081829071045, 2.5964674949645996, 2.6737515926361084, 2.7135961055755615, 2.672537326812744, 2.6894683837890625, 2.0877809524536133, 2.6409401893615723, 2.6512415409088135, 2.73057222366333, 2.6470141410827637, 2.6505703926086426, 2.540426015853882, 2.6932876110076904, 1.6765077114105225, 2.66463041305542, 2.684285879135132, 2.519714832305908, 2.521512269973755, 2.575150728225708, 2.684483051300049, 2.6053402423858643, 2.6341116428375244, 1.6438273191452026, 2.687291145324707, 0.2918194830417633, 2.671402931213379, 2.3155627250671387, 2.658900260925293, 2.683872699737549, 2.611534357070923, 1.6429033279418945, 2.26997709274292, 2.650719165802002, 2.5425195693969727, 2.662334442138672, 2.680142402648926, 2.636566162109375, 2.616905450820923, 1.645260214805603, 2.6472299098968506, 1.9968676567077637, 2.6431820392608643, 1.6484090089797974, 2.6562092304229736, 2.6928205490112305, 2.667092800140381, 2.6183300018310547, 2.7062947750091553, 2.7060840129852295, 2.646468162536621, 2.6623268127441406, 2.5934455394744873, 2.618043899536133, 2.490384101867676, 2.6259400844573975, 0.2951870262622833, 2.594383716583252, 2.5276217460632324, 2.6343164443969727, 0.2996112108230591, 2.683586359024048, 2.678327798843384, 2.5153603553771973, 2.7182106971740723, 2.6394248008728027, 1.643620491027832, 2.6797239780426025, 2.7298808097839355, 2.5934412479400635, 2.541983127593994, 2.6308224201202393, 2.682697057723999, 2.6754817962646484, 2.72536039352417, 2.690721035003662, 1.6436779499053955, 2.6333532333374023, 2.6870532035827637, 1.6467761993408203, 1.6417992115020752, 1.6623730659484863, 2.6369521617889404, 2.6817522048950195, 1.7103818655014038, 2.642019510269165, 2.691467046737671, 2.6168479919433594, 1.6544197797775269, 2.6346888542175293, 2.6458334922790527, 2.561271905899048, 1.6287858486175537, 2.6846299171447754, 2.673870086669922, 2.652771472930908, 2.696323871612549, 2.6316237449645996, 2.6678805351257324, 2.6384031772613525, 0.2894696593284607, 2.6753878593444824, 0.2904757261276245, 2.613978862762451, 2.615227460861206, 2.687108278274536, 2.5015006065368652, 2.6713898181915283, 0.2600633203983307, 0.3368908166885376, 2.5578784942626953, 2.6339871883392334, 1.6412224769592285, 2.633758306503296, 2.683535099029541, 2.7145726680755615, 2.652880907058716, 2.701887369155884, 2.68530011177063, 2.5249743461608887, 2.649555206298828, 2.711568832397461, 1.7089475393295288, 2.5620315074920654, 2.680983304977417, 2.662234306335449, 2.6848905086517334, 1.6475720405578613, 2.695446729660034, 2.6591877937316895, 2.662066698074341, 2.6290435791015625, 2.6522419452667236, 2.701390504837036, 2.6981358528137207, 0.30955371260643005, 1.6528751850128174, 2.5258612632751465, 2.6655144691467285, 1.6490527391433716, 0.2551635801792145, 1.6763509511947632, 2.6572747230529785, 2.7396702766418457, 2.6749489307403564, 2.7363877296447754, 2.6864237785339355, 2.6972296237945557, 2.4958181381225586, 2.4262895584106445, 1.636258840560913, 2.3058173656463623, 1.6549992561340332, 2.600590229034424, 2.6527559757232666, 2.614271640777588, 1.6633003950119019, 1.6122000217437744, 2.651601791381836, 2.6560909748077393, 1.6214771270751953, 2.640993118286133, 2.647740125656128, 2.709537982940674, 2.5839316844940186, 1.6473044157028198, 1.644559621810913, 0.29235127568244934, 2.6666455268859863, 0.26714956760406494, 2.675476312637329, 2.684635639190674, 2.5038528442382812, 2.3519582748413086, 2.6743900775909424, 2.529484272003174, 2.5232484340667725, 0.3278205990791321, 2.6553516387939453, 2.678508758544922, 0.27626127004623413, 1.6701545715332031, 2.509180784225464, 2.6679067611694336, 2.6297690868377686, 2.6631393432617188, 2.6412670612335205, 2.655750036239624, 1.6459534168243408, 2.4853289127349854, 2.662619113922119, 0.2987680435180664, 0.29665830731391907, 2.5048913955688477, 2.637247323989868, 2.509887218475342, 2.671630620956421, 2.4945285320281982, 2.5238325595855713, 1.683140754699707, 1.6444182395935059, 2.6859002113342285, 2.668423652648926, 1.6477257013320923, 2.6439411640167236, 2.4957563877105713, 2.5220463275909424, 1.7039154767990112, 2.608090877532959, 2.6305489540100098, 1.650476098060608, 2.667525291442871, 1.641448736190796, 2.6895313262939453, 1.6606091260910034, 0.284833163022995, 2.494249105453491, 0.31733226776123047, 0.32003289461135864, 1.7063512802124023, 0.2926914095878601, 2.6889731884002686, 2.614908456802368, 2.6432297229766846, 2.656972885131836, 2.6363627910614014, 2.691837787628174, 0.27223071455955505, 0.2949601709842682, 2.6600751876831055, 1.6199274063110352, 2.646472692489624, 2.661569833755493, 2.591729164123535, 2.6208980083465576, 0.2531299591064453, 2.653531551361084]
